In [2]:
import pandas as pd
import numpy as np
import tensorflow
from matplotlib import pyplot as plt
import datetime as dt
from pandas import concat as cot
from math import sqrt
from numpy import concatenate
from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
from sklearn.metrics import mean_squared_error, mean_absolute_error
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Reshape
from tensorflow.keras.layers import LSTM
from sklearn.preprocessing import FunctionTransformer
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import r2_score
from tensorflow.keras import layers, Model, utils
from tensorflow.keras.callbacks import TensorBoard, EarlyStopping
from numpy import median
from numpy import mean
import chinese_calendar

data = pd.read_csv('../../data/originalDataRegion.csv')

In [3]:
data['Date'] = pd.date_range('2020-12-15', '2022-01-16')
start_time = dt.date(2020, 12, 15)
end_time = dt.date(2022, 1, 16)

# Holiday feature boolean(add)
holiday_list = chinese_calendar.get_holidays(start_time, end_time)
data['is_holiday'] = np.where(data['Date'].isin(holiday_list), 1, 0)

# IssuePoint feature boolean
data['V1_IssuePoint'] = np.where(data['V1_IssuePoint'].notna(), 1, 0)
data['V2_H7_IssuePoint'] = np.where(data['V2_H7_IssuePoint'].notna(), 1, 0)
data['V3_IssuePoint'] = np.where(data['V3_IssuePoint'].notna(), 1, 0)
data['VacCategory'] = np.where(data['VacCategory'].notna(), 1, 0)

data['diff_VacDailySmoothed'] = data['VacDailySmoothed'].diff(1)

for i in [7, 14]:
	w = int(i/7)
	data['{}w_lagVacDailySmoothed'.format(w)] = 0
	for j in range(1, i+1):
		data['{}w_lagVacDailySmoothed'.format(w)] += data['VacDailySmoothed'].shift(j)

data['deltaVacWeekly'] = data['1w_lagVacDailySmoothed'].diff(periods=7)
deltaVacWeekly = np.array(data['deltaVacWeekly'])
where_are_inf = np.isinf(deltaVacWeekly)
where_are_nan = np.isnan(deltaVacWeekly)
deltaVacWeekly[where_are_inf] = 1
deltaVacWeekly[where_are_nan] = 0

data['deltaVacWeekly'] = deltaVacWeekly

def exponential_decay(t, init=0.5, m=6, finish=0.4):
    alpha = np.log(init / finish) / m
    l = - np.log(init) / alpha
    decay = np.exp(-alpha * (t + l))
    return decay
for i in data.columns[1:40].values:
	data[i] = \
		(data[i].shift(1) * exponential_decay(t=0) + \
		data[i].shift(2) * exponential_decay(t=1) + \
		data[i].shift(3) * exponential_decay(t=2) + \
		data[i].shift(4) * exponential_decay(t=3) + \
		data[i].shift(5) * exponential_decay(t=4) + \
		data[i].shift(6) * exponential_decay(t=5) + \
		data[i].shift(7) * exponential_decay(t=6))/7

data.set_index('Date', inplace=True)
data = data.drop(columns='index')
#data = data.fillna(0)

pdata = data.copy()
pdata = pdata.drop(columns=['new_deaths', 'new_deaths_smoothed', 'weekly_cases', 'biweekly_cases', 'daily_ people_1_dose',     							'total_vaccinations', 'people_1_dose', 'people_2_doses', 'people_boosters', 'people_vaccinated_per_hundred',
					'people_fully_vaccinated_per_hundred', 'total_boosters_per_hundred'])
pdata = pdata.fillna(0)

pdata['mean_7d_reproduction_rate'] = (pdata['reproduction_rate'].shift(1) + \
									  pdata['reproduction_rate'].shift(2) + \
									  pdata['reproduction_rate'].shift(3) + \
									  pdata['reproduction_rate'].shift(4) + \
									  pdata['reproduction_rate'].shift(5) + \
									  pdata['reproduction_rate'].shift(6) + \
									  pdata['reproduction_rate'].shift(7))/7

pdata['is_sensitive'] = np.where(pdata['mean_7d_reproduction_rate'] <= 1.0, 0, 1)

pdata['vac_aggressive'] = np.where((((pdata['is_sensitive'] == 1)&(pdata['new_cases'].shift(1) >= 30)&(pdata['new_cases'].shift(1) <= 300)) | ((pdata['is_sensitive'] == 0)&(pdata['new_cases'].shift(1) >= 60)&(pdata['new_cases'].shift(1) <= 300))), 1, 0)

In [ ]:

def series_to_supervised(data, n_in=1, span=-1, n_out=1, dropnan=True):
	n_vars = 1 if type(data) is list else data.shape[1]
	df = pd.DataFrame(data)
	cols, names = list(), list()

	for i in range(n_in, 0, -span):
		cols.append(df.shift(i))
		names += [('var%d(t-%d)' % (j+1, i)) for j in range(n_vars)]

	for i in range(0, n_out, span):
		cols.append(df.shift(-i))
		if i == 0:
			names += [('var%d(t)' % (j+1)) for j in range(n_vars)]
		else:
			names += [('var%d(t+%d)' % (j+1, i)) for j in range(n_vars)]

	agg = cot(cols, axis=1)
	agg.columns = names

	if dropnan:
		agg.dropna(inplace=True)
	return agg

daily_scaler = MinMaxScaler(feature_range=(0, 1))
delta_scaler = MaxAbsScaler()

train_scaler = MaxAbsScaler()
pdata['VacDailySmoothed'] = daily_scaler.fit_transform(pdata[['VacDailySmoothed']])
pdata['deltaVacWeekly'] = delta_scaler.fit_transform(pdata[['deltaVacWeekly']])


scaled_to_trans = pdata.iloc[:, np.r_[1, 8, 9, 40:45, 50, 57]]

temp_col = scaled_to_trans.VacDailySmoothed
scaled_to_trans = scaled_to_trans.drop('VacDailySmoothed',axis=1)
scaled_to_trans.insert(0,'VacDailySmoothed', temp_col)

scaled = train_scaler.fit_transform(scaled_to_trans)

scaled = scaled.astype('float32')

n_days = 7
span = 1
n_after = 1
n_features = 10
input_sample = int(n_days/span)

reframed = series_to_supervised(scaled, n_days, span, n_after)

In [17]:
ts_train_1 = pdata.truncate(before='2020-12-15', after='2021-04-11')
ts_train_2 = pdata.truncate(before='2020-12-15', after='2021-07-01')
ts_train_3 = pdata.truncate(before='2020-12-15', after='2022-12-08')

ts_1 = pdata.truncate(before='2020-12-15', after='2021-04-25')
ts_2 = pdata.truncate(before='2020-12-15', after='2021-07-15')
ts_3 = pdata.truncate(before='2020-12-15', after='2021-12-22')

ts_test_1 = pdata.truncate(before='2021-04-18', after='2021-04-25')
ts_test_2 = pdata.truncate(before='2021-07-08', after='2021-07-15')
ts_test_3 = pdata.truncate(before='2021-12-15', after='2021-12-22')

t1 = pdata.truncate(before='2020-12-15', after='2021-02-01')
t2 = pdata.truncate(before='2020-12-15', after='2021-06-15')
t3 = pdata.truncate(before='2020-12-15', after='2021-06-15')
t4 = pdata.truncate(before='2020-12-15', after='2021-08-15')

values = reframed.values
train_scene = values[len(t1):len(ts_2)+60, :]
test_scene = values[len(ts_3)-45:, :]

n_obs = input_sample * n_features
#s2 as train and s3 as test
train_X, train_y = train_scene[:, :n_obs], train_scene[:, -n_features]
test_X, test_y = test_scene[:, :n_obs], test_scene[:, -n_features]

print(train_X.shape, len(train_X), train_y.shape)
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], input_sample, n_features))
test_X = test_X.reshape((test_X.shape[0], input_sample, n_features))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)

(224, 70) 224 (224,)
(224, 7, 10) (224,) (63, 7, 10) (63,)


In [ ]:
#training

def mape(y_true, y_pred):
    mask=y_true!=0
    return np.fabs((y_true[mask]-y_pred[mask])/np.clip(y_true[mask],0.1,1)).mean()

def smape(y_true, y_pred):
    return 2.0 * np.mean(np.abs(y_pred - y_true) / (np.abs(y_pred) + np.abs(y_true)))

log_dir="logs/fit/" + dt.datetime.now().strftime("%Y%m%d-%H%M%S")
model_dir="my_model/train/" + dt.datetime.now().strftime("%Y%m%d-%H%M%S")
tbCallBack = TensorBoard(log_dir=log_dir,update_freq='epoch', histogram_freq=1,write_graph=True, write_images=True)
earlyStop = EarlyStopping(monitor="val_loss", verbose=2, mode='min', patience=3)
callbacks = [tbCallBack]
repeats = 20
error_scores, adjusted_r2scores, mse_scores, mae_scores, mape_scores, smape_scores, bestModelList = list(), list(), list(), list(), list(), list(), list()

inv_yhat_mean = np.array(pd.DataFrame(columns=range(len(test_y))))
inv_yhat_mean = 0
batch_size = 16
epochs = 100

for r in range(repeats):
    
    model = Sequential()

    model.add(LSTM(50, input_shape=(train_X.shape[1], train_X.shape[2])))

    model.add(Dropout(0.2))
    model.add(Dense(1))
    
    model.compile(loss='mse', optimizer='adam', metrics= ['accuracy'])

    print('train...')

    model.fit(train_X, train_y, epochs=epochs, batch_size=batch_size, validation_data=(test_X, test_y), verbose=1, shuffle=False, callbacks=callbacks)

    print('predict...')
    yhat = model.predict(test_X)
    test_X_p = test_X.reshape((test_X.shape[0], input_sample * n_features))

    inv_yhat = concatenate((yhat, test_X_p[:, -9:]), axis=1)
    inv_yhat = train_scaler.inverse_transform(inv_yhat)
    inv_yhat = inv_yhat[:,0]

    test_y_p = test_y.reshape((len(test_y), 1))
    inv_y = concatenate((test_y_p, test_X_p[:, -9:]), axis=1)
    inv_y = train_scaler.inverse_transform(inv_y)
    inv_y = inv_y[:,0]

    rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
    mse = mean_squared_error(inv_y, inv_yhat)
    mae = mean_absolute_error(inv_y, inv_yhat)
    mape_value = mape(inv_y, inv_yhat)
    smape_value = smape(inv_y, inv_yhat)

    if rmse < 0.05:
        inv_yhat_mean += np.array(inv_yhat)
        bestModelList.append(r+1)
        model.save("../../results/" + model_dir + "/%d_%d_rmse%.3f" % (r+1, repeats, rmse))

    n = (train_X.shape[0] + test_X.shape[0])
    Adjusted_R2 = 1-(((1-r2_score(inv_y,inv_yhat))*(n-1))/(n-n_features-1))
    print('%d) Test RMSE: %.3f' % (r+1, rmse))
    print('%d) Test Adjusted_R2: %.3f' % (r+1, Adjusted_R2))
    # MAE
    print('%d) Test MAE: %.3f' % (r+1, mae))
    # MSE
    print('%d) Test MSE: %.3f' % (r+1, mse))
    # MAPE
    print('%d) Test MAPE: %.3f' % (r+1, mape_value))
    # SMAPE
    print('%d) Test SMAPE: %.3f' % (r+1, smape_value))

    error_scores.append(rmse)
    adjusted_r2scores.append(Adjusted_R2)
    mae_scores.append(mae)
    mse_scores.append(mse)
    mape_scores.append(mape_value)
    smape_scores.append(smape_value)

results = pd.DataFrame()
results['error_scores'] = error_scores
results['adjusted_r2scores'] = adjusted_r2scores
results['mae_scores'] = mae_scores
results['mse_scores'] = mse_scores
results['mape_value'] = mape_scores
results['smape_value'] = smape_scores
print(results.describe())
results.to_csv('../../results/experiment_stateful.csv', index=False)

In [ ]:
inv_yhat_mean_display = inv_yhat_mean / len(bestModelList)
results = pd.read_csv('../../results/experiment_stateful.csv', header=0)

In [ ]:
relative_error = 0.
for i in range(len(test_y)):
    relative_error += (abs(inv_yhat_mean_display[i] - test_y[i]) / test_y[i]) ** 2
acc = 1- sqrt(relative_error / len(test_y))
fh = open('../../results/accuracy.txt', 'w', encoding='utf-8')
fh.write(f'Accuracy of model: {acc*100:.2f}%')
fh.close()

In [ ]:
from lime import lime_tabular

explainer = lime_tabular.RecurrentTabularExplainer(train_X,
                                                feature_names=['VacDailySmoothed', 'V1_5-15', 'V1_Clinical', 'V1_Crowded', 'V2_H7_IssuePoint', 'V3_IssuePoint', 'VacCategory','CHI', 'is_holiday', 'vac_aggressive'], 
                                                verbose=True, 
                                                mode='regression')
i = 40
exp = explainer.explain_instance(test_X[i], model.predict, num_features=10)
import json
import os
def write_list_to_json(list, json_file_name, json_file_save_path):
    os.chdir(json_file_save_path)
    with open(json_file_name, 'w') as  f:
        json.dump(list, f)

write_list_to_json(exp.as_list(), 'lime_results.json', '../../results/')